### Object Detection 
Object detection has been a core computer vision problem from the begining. The idea in object detection is that we again start with an input image and we have a **fixed** set of categories. We need to detect these categories from the image. The idea is every time the categories appear in the input image we want to draw a box around it and predict the label. This is challenging task because we don't know how many object will appear in the input image.

In object localization the number of object is fixed but here there may be different number of objects therefore we can address object detection with single object as the localization.



![alt text](images/od1.PNG)


In single object detection problem we only need to output 4 numbers(co-ordinates) of the object. But in multiple object detection we need to output 4 co-ordinates for each object. For example, if an image contain 3 objects say three dogs or whatever then we will have to output 12 numbers.

![alt text](images/od3.PNG)

### Approaches

##### **Approach-1: Sliding window**
 In this approach we will take an images and break it down into many patches and feed them into convolutional neural network. The network will output the class of each patch. In addition to the label we also want another label(background) from the model for each patch. This will confirm us that there is no object in a certain patch for a corresponding position in the image.

 **How do we choose these crops?**
 This has been a very challenging problem. Because,
 - there could be any number of objects in the image. 
 - The object could be in any position, of any size, and of any spatial size.

**Brute Force sliding**: If we apply brute force sliding window approach then we could end up with many many thousand crops. Then we need to feed each of them in the gigantic Neural networks which would be computationally intractable.

![alt text](images/od2.PNG)
   
**Region Proposals**: Instead of brute force sliding, we do have a traditional computer vision techniques named **region proposal network** which uses traditional signal/image processing techniques to list some proposals. The region proposal network will look into the blobby regions in the input image then give us some set of candidate proposal regions where objects might be potentially found. These methods are fast.
One region proposal method is **selective search** which give us 2 thousand proposals where the objects might be found. So, most of them won't be object rather nosies. But the precision of getting object is high.

![alt text](images/od4.PNG)



### Training
**R-CNN**

Rather than applying the classification network to every possible pixel of the image, we can first apply this region proposal network to the image. Then we can apply the classification model only to these proposed regions. This will be then computationally tractable. This idea was introduced in a paper **R-CNN**. The proposals are sometimes called "region of interests".

One problem with this method is that the regions could be of any size in the input image but our convolutional network need same size of inputs due to fully connected layers. Therefore we will take each of the proposals and warp them into a fixed size to be suited for the downstream network. Then we will run these warped proposals through CNN which will classify each of the proposed regions. In addition to that the R-CNN will also predict a regression to correct the bounding box. Because the regions might not be perfetc for an object in the input image. Therfore the network will predict four numbers that are kind of offset or a correction to the box that was predicted at the region proposal state. Then again this is a multitask loss in this case.

**Q** Are the region proposals learned? 

No. Because the region proposal network is a fixed function.

**Porblem**: Again this method is computationally expensive. Because we need to fed those 2k proposals into the Convolutional network. Moreover, the proposal regionss are fixed, we are not learning them. This is also a problem.


![alt text](images/rcnn.PNG)

**Fast R-CNN**

In fast R-CNN we first feed the input image through a CNN and get a high resolution feature map. Now rather than cropping iamge, based on the region proposals directly we projet the region proposals onto the feature map then taking the crops. This allow us to reuse a lot of expensive convolutional computation across the entire image when we have many many crops per image.

Then again we reshape the crops into same shape for running thorugh downstream network. We do this with the help of **RoI** pooling layer. Then we can run these reshaped crops through some fully connected layer to get class scores and regression offsets to the bounding boxes. When we train this, we again have a multitask loss that trades off between these two constraints and during back propagation we can back prop through this entire thing jointly.


![alt text](images/fast-rcnn.PNG)

**RoI pooling**: It is kind of Max Pooling layer. Here we won't discuss about much. For now I am just adding some images of steps of RoI...

![alt text](images/od5.PNG) ![alt text](images/od6.PNG)
![alt text](images/od7.PNG) ![alt text](images/od8.PNG)
![alt text](images/od9.PNG)

Fast R-CNN is about 10 times faster than R-CNN. The reason “Fast R-CNN” is faster than R-CNN is because you don’t have to feed 2000 region proposals to the convolutional neural network every time. Instead, the convolution operation is done only once per image and a feature map is generated from it.

The computation time of all the regions using selective search is about 2 seconds. Now since we train these regions in a shared way by sharing convolutions across entire image that can process all these region proposals in less than a second together. So fast R-CNN is bottlenecked by the computing of these region proposals. **Faster R-CNN** solve this problem.

**Faster R-CNN**
In fast R-CNN, the problem was that we had fixed region proposals. In faster R-CNN we will make the network to predict these regions itself. Again we will take an input image, pass through a CNN and get a feature map. Now we will use a separate region proposal network(a convolutional network) which will work on top of the feature map and predicts its region proposals. The rest of the process is same as fast R-CNN.

In faster R-CNN the network performs four task at once. Region proposal network predicts if there is an object or not and the bounding box co-ordinates for each of the proposals. Moreover the final network at the end needs to do these two things again. The final class scores and a second round of bounding box regression to again correcting errors that may have come from regioned proposal state.




There are two modules in faster R-CNN. One is RPN(Region Proposal Network) and the other one is "fast R-CNN" for detecting objects in the proposed regions.

Faster R-CNN workflow:
- RPN generates region proposals
- for each region proposal a fixed size feature vector is obtained using RoI(Region of Interest) pooling layer.
- These vectors are then classified using the fast R-CNN

![alt text](images/od10.PNG) ![alt text](images/od11.PNG)

#### Region Proposal Network(RPN)

The idea of region proposal network is that it propose/select regions on its own rather than fixed regions obtained by selective search.

- The region proposals are generated using a network that could be trained and customized according to the detection task
- Now since the regions are generated using a network, this can be trained end-to-end to be customized on the detection task.
- The RPN processes the image using the same convolutional layers used in the fast R-CNN detection network. Therefore, RPN doesnot take extra time to produce the regions.
- Due to the sharing the same convolutional layers, the RPN and the Fast R-CNN can be merged into a single network. Thus training is done once.

#### Anchor

The RPN takes the output feature map from the convolutional feature map. Then a sliding window(of size nXn) is passed through the feature map. For each window k regions are generated. These regions are parameterized by some reference boxes(regions within a region proposals) which are called **Anchor** boxes. Two parameters:
- Scale
- Aspect Ratio

Generally there are 3 scales and 3 aspect ratios and therefore 3*3=9 anchor boxes/regions for each region proposals. However all these regions will not put value into object detection. The regions will be selected based on the "objectness score"

![alt text](images/anchors.PNG)

For each region proposal a feature vector of same shape is obtained using RoI then fed to two sibling fully connected layers.
- First one generates binary classification/ objectness score. Which tell us if the region corresponds to a background or object.
- The other one returns a 4-D vectors defining the bounding box of the region.

**Q**: How do make the region proposal network learn when we don't have ground truth region proposals for the region proposal network?

**Ans**: The idea is that at any time we have a regioned proposal which has more than some threshold of overlap with the ground truth objects then we consider that region as positive and predict that region as a proposal. When we have less overlap we consider that region as negative don't mark as a region. The threshold value is determined by objectness score.

#### objectness Score
We know that the first layer returns a binary classification scores. If the first one is 1 and second one is zero then it refers to a background and vice versa.

For training the RPN each anchor is given a positive or negative objectness based on the Intersection-over-Union(IoU).
**IoU**:
   - ratio between the area of intersection between the anchor box and ground box to the area of union of the two boxes.

**IoU** Evaluation: Aparantly, the IoU ranges from 0 and 1.
Objectness sign is determined based on the below conditions.
- IoU greater than equal .5 then positive.
- IoU less than .3 then negative
- IoU greater than equal .3 and less than equal .5 then neither positive nor negative.


![alt text](images/obj.PNG)

**Q**: Whata is the classification loss at the region proposal network?

**Ans**: Again, this self learning region proposal network is a complex one. We might talk about later. But it's making binary classification decision on some set of regions it's considering. If the region is an object or not. Therefore we will use binary classification loss.

#### Summary of faster R-CNN
The below picture sums up the whole procedure of faster R-CNN for object detection

![alt text](images/conclusion.PNG)

So far we have discussed different types of R-CNN. Here "R" stands for region. That means all the methods are region based obeject detection method. There are other methods available for object detection. Such as,
- YOLO(You Only Look Once)
- SSD(Single Shot Detection)

#### YOLO/SSD/RetinaNet

Although the faster R-CNN works fine it has drawbacks. The model doesnot look at the whole image rather it takes some of the positions from the image where objects can be found. To solve this issue we use sophisticated model "YOLO"(You only look once).

The idea is that rather than doing independent processing for each of these potential regions, instead we want to try and treat this like a regression problem and just make all these predictions all at once with some big convolutional network. 

**Overview**: We take our input image and divide it into some course of grid(in our example it's 7x7). Then for each of the grid we imagine some set of the base bounding boxes. Here we have taken 3(a tall, a wide, and a square) but in practice we will take more than 3.

Now we want to predict an offset for each of the base bounding boxes to predict the location of the object of this base bounding box and a classification score for each of the boxes. So we end up predicting a giant tensors of 7x7x(5B+C). We have B based bounding boxes and each has 5 numbers, the offset(co-ordinates:4 numbers) and the classification score for that box and finally C classification scores for C categories. So the problem is now, we have an input image and we get a 3 dimensional output by using a giant CNN.


![alt text](images/od12.PNG)

The way the region proposal network works in faster R-CNN is quite similar of this YOLO method. There were some base bounding boxes over some grid of image and region proposal network does some classification and regression.

#### YOLO Workflow
- Takes an input and divides it into grids (say 3X3). In practical we will use more grids in order to get better predictions.
- Classification and localization are applied to each grid.

**Dataset Type**: To train our model we need labelled data.
If we divide an image into 3X3 grid, number of classes is 3(say, pedestrain, car,motorcycle respectivelly), then the output will be a 8 dimensional vector.

|y| Description|
|---|----------|
|pc| pc defines whether an object is present in the grid or not (it is the probability)|
|bx|x coordinate of the midpoint of the object|
|by|y coordinate of the midpoint of the object|
|bh|ratio of the height of the bounding box to the height of the corresponding grid cell/regions |
|bw|ratio of the width of the bounding box to the width of the corresponding grid cell/regions |
|c1|class 1|
|c2|class 2|
|c3|class 3|



![alt text](images/y1.PNG) ![alt text](images/y2.PNG)

let's say we select the first grid marked in the lime color. Since there is no object in the grid the pc will be 0 and the other values will be "?" which maens the values don't matter.
If any grid contains object then pc will be 1 and the bx,by,bh,bw will be computed according the coordinates of the bounding box/grid. Say we are working with the grid colored with aqua.

![alt text](images/y3.PNG)
 
then c2 will be 1 referring to the car class. Now to compute bx,by,bh,bw let's consider the center-right grid which contains a car. In YOLO, the coordinates assigned to all the grids are:

![alt text](images/y4.PNG)![alt text](images/y5.PNG)

So, bx will be around .4 and by is .3, bh=.9, bw=.5(approximately). So, the y labels for this grid will be, 1,.4,.3,.9,.5,0,1,0. Notice the bh and bw will always ranges between 0 and 1 since the mid point will always lies within the grid.Whereas bh and bw can be more than 1 in case the dimensions of the bounding box are more than the dimension of the grid.

Since for each grid the output vector is of shape 8 the final output shape for the image will be of size 3X3X8. The input image is: 100X100X3 and the output will be 3X3X8. Our model will be trained as follows

![alt text](images/y6.PNG)

**Note**
- although an object spans out to more than one grid it will be assigned to a single grid in which its mid point is located.
- We can reduce the chances of multiple objects appearing in the same grid cell by increasing the more number of grids 

##### Improving the detection performance
**IoU**
How do we know if the prediction of our model is good or bad. To evaluate model's performance we use IoU. This is a ratio between the intersection and the union between the predicted box and the acutual box. We set a threshold value for IoU. If the IoU of our model is greater than the threshold thne we can consider good prediction. The higher the threshold is set the better detection will be output. In general problems it is set to .5

**Union and Non-Max Suppression**
One common problem in object detection is that a single object can be detected multiple times. To overcome this we apply Non-Max Suppression.

**Approach**: It looks all the probabilities for a single object and takes the highest one and discard the others.
- 1 Discard all the boxes having probabilities less than or equal to a pre-defined threshold (say, 0.5)
- 2 For the remaining boxes:
   - Pick the box with the highest probability and take that as the output prediction
   - Discard any other box which has IoU greater than the threshold with the output box from the above step
- Repeat step 2 untill all the boxes are either taken as the output prediction or discarded.

There is another method named "anchor boxes" which improves the alogorithm.


**Anchor Boxes**:
So far, we have seen that one grid can detect one object. However in some cases where the mid points of more than one objects are located in the same grid. Therefore we need to detect all the object for better performance. Here comes the role of anchor boxes.

We already discussed that according to the aspect rations and scale there can be many anchor boxes within a single grid/region proposal. Therefore, based on the different size,shape of the anchor boxes, different object is detected within a grid.

 ![alt text](images/yy7.PNG) ![alt text](images/y8.PNG)

Here in our example we will define two anchor boxes of different shapes. One for car and the other for the person. Then the model can detect both objects correponding to the anchor boxes.

well, if we have more than one anchor boxes then the shape of output vector will be different. For a single anchor box the output is of size 8. So for n anchor boxes this size will be n*8. For gird size 3, number of  classes 3, anchor box n=2, then the output will be of shape 3X3Xn*8=3X3X2*8=3X3X16. So we can generalize the output as , 

output= grid_size X grid_size X (5+num_classes)*num_anchor boxes


#### Testing
During testing we will divide the image with the same number of grids which we have choosen in the training time. After getting the output we will apply Non-Max-Suppression technique.

Object detection has lots of variables. Moreover, there can be different base networks. There is also hybrid method called **RFC** where the techniques from region-based detection and YOLO based detection are blended.

Therer are different hyperparameters e.g. what is the image size, how many region proposals do we need etc.. New works have done on object detection like **CPR**.

![alt text](images/od13.PNG)


Once we have gainde these ideas, we can apply more than one techniqes in particular problems. For example we can apply object detection and captioning together in a same problem. Where now we want to classify each of the bounding box as well as a caption corresponding to the object in that bounding box.

**References**
1. http://cs231n.stanford.edu/slides/2017/
2. https://blog.paperspace.com/faster-r-cnn-explained-object-detection/
3. https://www.analyticsvidhya.com/blog/2018/12/practical-guide-object-detection-yolo-framewor-python/?utm_source=blog&utm_medium=a-step-by-step-introduction-to-the-basic-object-detection-algorithms-part-1